In [1]:
import pandas as pd

In [ ]:
import pandas as pd
import ast

# -------------------------
# STEP 1: Load the CSV
# -------------------------
df = pd.read_csv('job_postings_burning_glass.csv') 

# -------------------------
# STEP 2: Define Columns to Combine
# -------------------------
skill_columns = [
    "SPECIALIZED_SKILLS_NAME",
    "CERTIFICATIONS_NAME",
    "COMMON_SKILLS_NAME",
    "SOFTWARE_SKILLS_NAME"
]

# -------------------------
# STEP 3: Convert stringified lists to actual lists and explode
# -------------------------
# Function to safely parse lists
def safe_parse_list(cell):
    if pd.isnull(cell):
        return []
    try:
        return ast.literal_eval(cell)
    except Exception:
        return []

# Apply and combine all skills into one column
skill_rows = []

for col in skill_columns:
    temp_df = df[['COMPANY_NAME', col]].copy()
    temp_df[col] = temp_df[col].apply(safe_parse_list)
    temp_df = temp_df.explode(col)
    temp_df.rename(columns={col: 'SKILLS_NAME'}, inplace=True)
    skill_rows.append(temp_df)

# Combine all skill records into one DataFrame
combined_skills_df = pd.concat(skill_rows, ignore_index=True)
combined_skills_df.dropna(subset=['SKILLS_NAME'], inplace=True)

# Normalize text
combined_skills_df['SKILLS_NAME'] = combined_skills_df['SKILLS_NAME'].str.strip().str.lower()

# -------------------------
# STEP 4: Define Core AI Skills
# -------------------------
core_ai_terms = ['machine learning', 'natural language processing', 'computer vision', 'artificial intelligence']

# Find if the skill itself is a core AI skill
combined_skills_df['IS_CORE_AI'] = combined_skills_df['SKILLS_NAME'].apply(
    lambda skill: any(term in skill for term in core_ai_terms)
)

# -------------------------
# STEP 5: Calculate AI-Relatedness Score
# -------------------------
# Denominator: total times each skill appears
total_counts = combined_skills_df['SKILLS_NAME'].value_counts()

# Numerator: co-occurs with core AI
ai_counts = combined_skills_df[combined_skills_df['IS_CORE_AI']]['SKILLS_NAME'].value_counts()

# AI-Relatedness = co-occurrence / total occurrences
ai_score = (ai_counts / total_counts).fillna(0)

# Final AI Score Table
ai_score_df = ai_score.reset_index()
ai_score_df.columns = ['SKILLS_NAME', 'AI_RELATEDNESS_SCORE']

# -------------------------
# STEP 6: Filter by Threshold and Map Back to Company
# -------------------------
threshold = 0.01
filtered_skills_df = ai_score_df[ai_score_df['AI_RELATEDNESS_SCORE'] >= threshold]

# Merge to find which companies had AI-related skills
final_df = combined_skills_df.merge(filtered_skills_df, on='SKILLS_NAME')

# Group AI-related skills per company
company_ai_skills = (
    final_df.groupby('COMPANY_NAME')['SKILLS_NAME']
    .unique()
    .reset_index()
    .rename(columns={'SKILLS_NAME': 'AI_RELATED_SKILLS'})
)

# -------------------------
# STEP 7: Save or Show Outputs
# -------------------------
print("Top 10 AI-related skills:\n", filtered_skills_df.sort_values(by='AI_RELATEDNESS_SCORE', ascending=False).head(10))
print("\nSample companies with AI-related skills:\n", company_ai_skills.head(10))



Top 10 AI-related skills:
           SKILLS_NAME  AI_RELATEDNESS_SCORE
782  machine learning                   1.0

Sample companies with AI-related skills:
               COMPANY_NAME   AI_RELATED_SKILLS
0  University of Minnesota  [machine learning]
